In [1]:
# %%writefile ../alignment_to_database.py

# """
# Script aims to read in the reference_dataframe file, select a taxonomic
# level and group, and read the path to the location of that data. It then
# prepares data for machine learning by converting base pair coding to numerical
# encoding, pads it out and then runs the algorithm
# """

# import pandas as pd
# from Bio import SeqIO
# import numpy as np
# import os
# import random
# import argparse
# import math
# import matplotlib.pyplot as plt
# import pandas as pd

# parser = argparse.ArgumentParser(description="""
# Script aims to read in the reference_dataframe file, and for each
# sample, randomly subsample n_reads number of reads and save the
# file, and save the keys used as as separate file. The script ends there.

# Outside the script these files will be utilised in minimap2 to
# map against a specified database, produce an output paf file
# and then remove the file to conserve space. 
# """)
# parser.add_argument("ref_df_fn", help="File path to the reference dataframe")
# parser.add_argument("data_root", help="Root folder for analysis/")
# parser.add_argument("--n_reads", "-c", help="count of reads per class")
# parser.add_argument("--d_type", "-d", help="type of database - CUSTOM or UNITE")
# group = parser.add_mutually_exclusive_group()
# group.add_argument("--verbose", "-v", "--v", action="store_true")
# group.add_argument("--quiet", "-q", "--q", action="store_true")
# args = parser.parse_args()

# # assign required arguments to variables
# ref_df_fn = args.ref_df_fn
# data_root = args.data_root

# # assign a number of reads per class
# n_reads = int(args.n_reads)

# # test to make sure both required file paths are input
# try:
#     os.path.exists(ref_df_fn)
# except:
#     print('Cannot find %s' % ref_df_fn)
# try:
#     os.path.exists(data_root)
# except:
#     print('Cannot find %s' % data_root)
    
# if args.verbose:
#     print('\033[1;34m' + "Reference dataframe is at " + ref_df_fn + '\033[0m')
#     print('\033[1;34m' + "Root directory is at " + data_root + '\033[0m')
#     print('\033[1;34m' + "Count of reads per sample is", n_reads,'\033[0m')
    
# # read in the reference dataframe from the argument path
# ref_df = pd.read_csv(ref_df_fn, index_col=None)

# # check whether the reference dataframe implies there are enough reads
# # to continue given n_reads
# try:
#     if ref_df[ref_df["# for use"] \
#               < n_reads].shape[0] > 0 :
#         print("These species need more reads.")
#         print(ref_df[ref_df["# for use"] \
#               < n_reads])
#         exit()
# except:
#     print('Check %s to have the wanted column names' % ref_df_fn)
   

# for index, row in ref_df.iterrows():
#     fasta_dict = SeqIO.to_dict(SeqIO.parse(row['path for use'], "fasta"))
#     key_list = []
#     for key in fasta_dict:
#         key_list.append(key)
#     print(len(key_list))
#     keys_list = random.sample(key_list,k=n_reads)
#     new_dict = {}
#     for key in keys_list:
#         new_dict[key] = fasta_dict[key]
#     if args.d_type.upper() == 'CUSTOM':
#         SeqIO.write(new_dict.values(), data_root+"database_mapping/custom/%s_%s.fasta" % (row['genus'],row['species']), "fasta")
#         with open(data_root+'database_mapping/custom/%s_%s_keys.csv'% (row['genus'],row['species']), 'w+') as f:
#             for key in keys_list:
#                 f.write("%s\n"%(key))
#     elif args.d_type.upper() == 'UNITE':
#         SeqIO.write(new_dict.values(), "analysis/database_mapping/unite/%s_%s.fasta" % (row['genus'],row['species']), "fasta")
#         with open(data_root+'database_mapping/unite/%s_%s_keys.csv'% (row['genus'],row['species']), 'w+') as f:
#             for key in keys_list:
#                 f.write("%s\n"%(key))

In [2]:
# %%writefile ../custom_minimap_result.py

# import Bio
# from Bio import SeqIO
# from Bio.Seq import Seq
# from Bio.SeqRecord import SeqRecord as SR
# from Bio.Blast import NCBIXML
# from Bio import pairwise2
# from Bio.pairwise2 import format_alignment
# from Bio import AlignIO
# from Bio.Align import AlignInfo
# import numpy as np
# import pandas as pd
# from pandas import DataFrame as df
# # import seaborn as sns
# import csv
# import matplotlib
# import matplotlib.pyplot as plt
# import subprocess
# import os
# from shutil import copy
# import random
# import warnings
# from Bio import pairwise2
# from Bio.pairwise2 import format_alignment
# # from ipysankeywidget import SankeyWidget
# warnings.filterwarnings("ignore")
# import argparse


# parser = argparse.ArgumentParser(description="""
# This program extracts a specified number of reads from a fasta
# file and saves them to a new fasta file
# """)
# group = parser.add_mutually_exclusive_group()
# group.add_argument("--verbose", "-v", "--v", action="store_true")
# group.add_argument("--quiet", "-q", "--q", action="store_true")
# parser.add_argument("input_file", help="The input file for extraction")
# args = parser.parse_args()

# if args.verbose:
#     print("Input file is " + args.input_file + "\n")

# f=open(args.input_file,"r")
# if f.mode == "r":
#     contents=f.read()

# tmp=contents.replace("\t",",").split('\n')
# tmp_dict = {}
# for line in tmp[:-1]:
#     tmp_dict[line.split(",")[0]] = str(line.split(",")[5])

# count_dict = {}
# correct = 0
# incorrect = 0
# for item in tmp_dict:
#     if tmp_dict[item] not in count_dict:
#         count_dict[tmp_dict[item]] = 1
#     else:
#         count_dict[tmp_dict[item]] = count_dict[tmp_dict[item]] + 1
        
# tmp = pd.DataFrame.from_dict(count_dict,orient='index',columns=["Count"])
# tmp.index.names = [args.input_file[33:-4]]
# tmp = tmp.sort_values(by="Count",ascending=False)

# if args.verbose:
#     print(tmp)

# tmp.to_csv(args.input_file[:-4]+'_match_distribution.csv',sep=',')

Starting points from Tavish  

reference_dataframe at '/media/MassStorage/tmp/TE/honours/analysis/Stats/reference_dataframe.csv' 

custom_database at '/media/MassStorage/tmp/TE/honours/database/custom_database_labelled.fasta'

taxonomy_file at '/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file.csv'

#### workflow

* Two databases
* subsample 15000 reads per each mock community species. Save those out.
* map reads against both databases with minimap safe out data in paf format.
* get best hit per species (see what this means while looking at the data).
* add the full taxonomy to each best match using the taxonomy file.
* summarize data at different taxonomic ranks for each species.
* pull this all together somehow (summary across all the samples? focus on species of interest e.g. deleted from analyis?)

In [3]:
from Bio import SeqIO
import os
import random
import subprocess
import pandas as pd

In [4]:
reference_dataframe_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/analysis/Stats/reference_dataframe.csv')
max_custom_database_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/database/custom_database_labelled.fasta')
taxonomy_file_fn = os.path.abspath('/media/MassStorage/tmp/TE/honours/analysis/Stats/taxonomy_file.csv')

In [5]:
INPUT_BASEDIR = os.path.abspath('/media/MassStorage/tmp/TE/honours')

OUT_DIR = os.path.abspath('../../analysis/Mapping')
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

In [6]:
### list of species in the max database
max_species = ['Puccinia_striiformis-tritici',
             'Zymoseptoria_tritici',
             'Pyrenophora_tritici-repentis',
             'Fusarium_oxysporum',
             'Tuber_brumale',
             'Cortinarius_globuliformis',
             'Aspergillus_niger',
             'Clavispora_lusitaniae',
             'Kluyveromyces_unidentified',
             'Penicillium_chrysogenum',
             'Rhodotorula_mucilaginosa',
             'Scedosporium_boydii',
             'Blastobotrys_proliferans',
             'Debaryomyces_unidentified',
             'Galactomyces_geotrichum',
             'Kodamaea_ohmeri',
             'Meyerozyma_guillermondii',
             'Wickerhamomyces_anomalus',
             'Yamadazyma_mexicana',
             'Yamadazyma_scolyti',
             'Yarrowia_lipolytica',
             'Zygoascus_hellenicus',
             'Aspergillus_flavus',
             'Cryptococcus_zero',
             'Aspergillus_unidentified',
             'Diaporthe_CCL067',
             'Diaporthe_unidentified',
             'Oculimacula_yallundae-CCL031',
             'Oculimacula_yallundae-CCL029',
             'Dothiorella_vidmadera',
             'Quambalaria_cyanescens',
             'Entoleuca_unidentified',
             'Asteroma_CCL060',
             'Asteroma_CCL068',
             'Saccharomyces_cerevisiae',
             'Cladophialophora_unidentified',
             'Candida_albicans',
             'Candida_metapsilosis',
             'Candida_orthopsilosis',
             'Candida_parapsilosis',
             'Candida_unidentified',
             'Kluyveromyces_marxianus',
             'Pichia_kudriavzevii',
             'Pichia_membranifaciens']

In [7]:
###Removed from second test database
species_delete = ['Candida_orthopsilosis',
                 'Candida_metapsilosis',
                 'Aspergillus_niger',
                 'Yamadazyma_mexicana']

In [8]:
###species to be searched against both databases
mock_community = ['Puccinia_striiformis-tritici',
             'Zymoseptoria_tritici',
             'Pyrenophora_tritici-repentis',
             'Fusarium_oxysporum',
             'Tuber_brumale',
             'Cortinarius_globuliformis',
             'Aspergillus_niger',
             'Clavispora_lusitaniae',
             'Kluyveromyces_unidentified',
             'Penicillium_chrysogenum',
             'Rhodotorula_mucilaginosa',
             'Scedosporium_boydii',
             'Blastobotrys_proliferans',
             'Debaryomyces_unidentified',
             'Galactomyces_geotrichum',
             'Kodamaea_ohmeri',
             'Meyerozyma_guillermondii',
             'Wickerhamomyces_anomalus',
             'Yamadazyma_mexicana',
             'Yamadazyma_scolyti',
             'Yarrowia_lipolytica',
             'Zygoascus_hellenicus',
             'Aspergillus_flavus',
             'Cryptococcus_zero',
             'Aspergillus_unidentified',
             'Diaporthe_CCL067',
             'Diaporthe_unidentified',
             'Oculimacula_yallundae-CCL031',
             'Oculimacula_yallundae-CCL029',
             'Dothiorella_vidmadera',
             'Quambalaria_cyanescens',
             'Entoleuca_unidentified',
             'Asteroma_CCL060',
             'Asteroma_CCL068',
             'Saccharomyces_cerevisiae',
             'Cladophialophora_unidentified',
             'Candida_albicans',
             'Candida_metapsilosis',
             'Candida_orthopsilosis',
             'Candida_parapsilosis',
             'Candida_unidentified',
             'Kluyveromyces_marxianus',
             'Pichia_kudriavzevii',
             'Pichia_membranifaciens']

In [9]:
ref_df = pd.read_csv(reference_dataframe_fn)
ref_df['name_species'] = ref_df['genus'] +"_"+ ref_df['species']

In [10]:
ref_df.name_species.tolist()

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'aspergillus_niger',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_mexicana',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_metapsilosis',


In [11]:
new_db_fn = os.path.join(OUT_DIR, 'gsref.db.fasta')

new_db_list = []
old_db_list = []
for seq in SeqIO.parse(max_custom_database_fn, 'fasta'):
    old_db_list.append(seq.id)
    if seq.id.lower() in ref_df.name_species.tolist():
        #print(seq.id)
        seq.id = seq.name = seq.description = seq.id.lower()
        new_db_list.append(seq)
    else:
        print(seq.id)

In [12]:
if len(new_db_list) == len(old_db_list):
    SeqIO.write(new_db_list, new_db_fn, 'fasta')
else:
    print("please check!")

In [13]:
sub_db_fn = os.path.join(OUT_DIR, 'gsref.subdb.fasta')
sub_db_list = []
for seq in new_db_list:
    if seq.id not in [x.lower() for x in species_delete]:
        sub_db_list.append(seq)

In [14]:
if len(sub_db_list) + len(species_delete) == len(new_db_list):
    SeqIO.write(sub_db_list, sub_db_fn, 'fasta' )
else:
    print("please check!")

In [15]:
[x.id for x in sub_db_list]

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_parapsilosis',
 'candida_unidentified',
 'kluyveromyces_marxi

In [16]:
mock_community = [x.lower() for x in mock_community]

In [17]:
mock_community

['puccinia_striiformis-tritici',
 'zymoseptoria_tritici',
 'pyrenophora_tritici-repentis',
 'fusarium_oxysporum',
 'tuber_brumale',
 'cortinarius_globuliformis',
 'aspergillus_niger',
 'clavispora_lusitaniae',
 'kluyveromyces_unidentified',
 'penicillium_chrysogenum',
 'rhodotorula_mucilaginosa',
 'scedosporium_boydii',
 'blastobotrys_proliferans',
 'debaryomyces_unidentified',
 'galactomyces_geotrichum',
 'kodamaea_ohmeri',
 'meyerozyma_guillermondii',
 'wickerhamomyces_anomalus',
 'yamadazyma_mexicana',
 'yamadazyma_scolyti',
 'yarrowia_lipolytica',
 'zygoascus_hellenicus',
 'aspergillus_flavus',
 'cryptococcus_zero',
 'aspergillus_unidentified',
 'diaporthe_ccl067',
 'diaporthe_unidentified',
 'oculimacula_yallundae-ccl031',
 'oculimacula_yallundae-ccl029',
 'dothiorella_vidmadera',
 'quambalaria_cyanescens',
 'entoleuca_unidentified',
 'asteroma_ccl060',
 'asteroma_ccl068',
 'saccharomyces_cerevisiae',
 'cladophialophora_unidentified',
 'candida_albicans',
 'candida_metapsilosis',


In [31]:
def subsamplereads(in_fn, out_fn, n_reads):
    command = F'reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}'
    out = subprocess.getstatusoutput(command)
    if out[0] == 0:
        print(F":)Completed {command}\n")
    else:
        print(F":(check one {command}!!\n")

In [32]:
n_reads = 15000

In [33]:
MC_READ_DIR = os.path.join(OUT_DIR, 'MC_READS')
if not os.path.exists(MC_READ_DIR):
    os.mkdir(MC_READ_DIR)

In [34]:
ref_df.columns

Index(['Unnamed: 0', 'species', 'genus', 'family', 'order', 'class', 'phylum',
       'kingdom', '# raw reads', '# reads after homology filtering',
       '# reads after length filtering', '# for use', 'path to raw reads',
       'path to homology filtering', 'path to length filtering',
       'path for use', 'name_species'],
      dtype='object')

In [35]:
fn_subsampling = {}
for x in mock_community:
    fn_subsampling[x] = (ref_df[(ref_df['species'] == x.split('_')[1]) & (ref_df['genus'] == x.split('_')[0])]['path for use'].tolist()[0])
    fn_subsampling[x] = os.path.join(INPUT_BASEDIR, fn_subsampling[x])
fn_subsampling

{'puccinia_striiformis-tritici': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode01/length_restricted_for_use.fasta',
 'zymoseptoria_tritici': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode02/length_restricted_for_use.fasta',
 'pyrenophora_tritici-repentis': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode03/length_restricted_for_use.fasta',
 'fusarium_oxysporum': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode04/length_restricted_for_use.fasta',
 'tuber_brumale': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode05/length_restricted_for_use.fasta',
 'cortinarius_globuliformis': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcode06/length_restricted_for_use.fasta',
 'aspergillus_niger': '/media/MassStorage/tmp/TE/honours/analysis/Length_Filtered/20171103_FAH15473/barcod

In [36]:
sub_reads_fn = {}
for key, value in fn_subsampling.items():
    species = key
    in_fn = value
    out_fn = os.path.join(MC_READ_DIR, F'{species}.{n_reads}.fasta')
    print(out_fn)
    subsamplereads(in_fn, out_fn, n_reads)
    sub_reads_fn[species] = out_fn
sub_reads_fn

/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/puccinia_striiformis-tritici.15000.fasta
:(check one reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}!!

/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/zymoseptoria_tritici.15000.fasta
:(check one reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}!!

/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/pyrenophora_tritici-repentis.15000.fasta
:(check one reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}!!

/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/fusarium_oxysporum.15000.fasta
:(check one reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}!!

/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/tuber_brumale.15000.fasta
:(check one reformat.sh samplereadstarget={n_reads} in={in_fn} out={out_fn}!!

/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/cortinarius_globuliformis.15000.fasta
:(check one reformat.sh samplereads

{'puccinia_striiformis-tritici': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/puccinia_striiformis-tritici.15000.fasta',
 'zymoseptoria_tritici': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/zymoseptoria_tritici.15000.fasta',
 'pyrenophora_tritici-repentis': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/pyrenophora_tritici-repentis.15000.fasta',
 'fusarium_oxysporum': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/fusarium_oxysporum.15000.fasta',
 'tuber_brumale': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/tuber_brumale.15000.fasta',
 'cortinarius_globuliformis': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/cortinarius_globuliformis.15000.fasta',
 'aspergillus_niger': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/aspergillus_niger.15000.fasta',
 'clavispora_lusitaniae': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/clavispora_lusitaniae.15000.fasta',
 'kluyveromy

### Map with minimap against both databases

In [24]:
def minimapmapping(fasta_fn, ref_fn, out_fn):
    command = F"minimap2 -x map-ont -t 6 {ref_fn} {fasta_fn} -o {out_fn}"
    out = subprocess.getstatusoutput(command)
    print(out)

In [25]:
dbases_fn = {}
for x in [sub_db_fn, new_db_fn]:
    dbases_fn[x] = os.path.join(OUT_DIR, os.path.basename(x).replace('.fasta', '').replace('.','_'))
    if not os.path.exists(dbases_fn[x]):
        os.mkdir(dbases_fn[x])
dbases_fn

{'/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb',
 '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db'}

In [26]:
db_fn = sub_db_fn
sub_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    tmp_out = dbases_fn[db_fn]
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(tmp_out, F"{db_name}.{species}.minimap2.paf")
    sub_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn)

(0, "[M::mm_idx_gen::0.011*1.18] collected minimizers\n[M::mm_idx_gen::0.013*1.97] sorted minimizers\n[M::main::0.013*1.97] loaded/built the index for 40 target sequence(s)\n[M::mm_mapopt_update::0.014*1.92] mid_occ = 41\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 40\n[M::mm_idx_stat::0.015*1.88] distinct minimizers: 9451 (73.11% are singletons); average occurrences: 2.301; average spacing: 5.425\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/puccinia_striiformis-tritici.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.puccinia_striiformis-tritici.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/puccinia_striiformis-tritici.15000.fasta\n[M::main] Real time: 0.016 sec; CPU: 0.029 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0

(0, "[M::mm_idx_gen::0.010*1.12] collected minimizers\n[M::mm_idx_gen::0.013*1.93] sorted minimizers\n[M::main::0.013*1.93] loaded/built the index for 40 target sequence(s)\n[M::mm_mapopt_update::0.014*1.88] mid_occ = 41\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 40\n[M::mm_idx_stat::0.014*1.84] distinct minimizers: 9451 (73.11% are singletons); average occurrences: 2.301; average spacing: 5.425\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.penicillium_chrysogenum.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta\n[M::main] Real time: 0.017 sec; CPU: 0.028 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.006*1.26] coll

(0, "[M::mm_idx_gen::0.010*1.12] collected minimizers\n[M::mm_idx_gen::0.012*2.07] sorted minimizers\n[M::main::0.012*2.07] loaded/built the index for 40 target sequence(s)\n[M::mm_mapopt_update::0.013*2.00] mid_occ = 41\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 40\n[M::mm_idx_stat::0.014*1.95] distinct minimizers: 9451 (73.11% are singletons); average occurrences: 2.301; average spacing: 5.425\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/yamadazyma_mexicana.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.yamadazyma_mexicana.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/yamadazyma_mexicana.15000.fasta\n[M::main] Real time: 0.016 sec; CPU: 0.029 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.010*1.15] collected minimi

(0, "[M::mm_idx_gen::0.009*1.18] collected minimizers\n[M::mm_idx_gen::0.012*2.08] sorted minimizers\n[M::main::0.012*2.08] loaded/built the index for 40 target sequence(s)\n[M::mm_mapopt_update::0.013*1.99] mid_occ = 41\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 40\n[M::mm_idx_stat::0.014*1.93] distinct minimizers: 9451 (73.11% are singletons); average occurrences: 2.301; average spacing: 5.425\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/oculimacula_yallundae-ccl029.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.oculimacula_yallundae-ccl029.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/oculimacula_yallundae-ccl029.15000.fasta\n[M::main] Real time: 0.016 sec; CPU: 0.029 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0

(0, "[M::mm_idx_gen::0.009*1.21] collected minimizers\n[M::mm_idx_gen::0.012*2.21] sorted minimizers\n[M::main::0.012*2.20] loaded/built the index for 40 target sequence(s)\n[M::mm_mapopt_update::0.013*2.09] mid_occ = 41\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 40\n[M::mm_idx_stat::0.014*2.02] distinct minimizers: 9451 (73.11% are singletons); average occurrences: 2.301; average spacing: 5.425\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/candida_metapsilosis.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.candida_metapsilosis.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.subdb.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/candida_metapsilosis.15000.fasta\n[M::main] Real time: 0.016 sec; CPU: 0.030 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.009*1.21] collected min

In [27]:
sub_db_mapping_fn

{'puccinia_striiformis-tritici': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.puccinia_striiformis-tritici.minimap2.paf',
 'zymoseptoria_tritici': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.zymoseptoria_tritici.minimap2.paf',
 'pyrenophora_tritici-repentis': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.pyrenophora_tritici-repentis.minimap2.paf',
 'fusarium_oxysporum': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.fusarium_oxysporum.minimap2.paf',
 'tuber_brumale': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.tuber_brumale.minimap2.paf',
 'cortinarius_globuliformis': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.cortinarius_globuliformis.minimap2.paf',
 'aspergillus_niger': '/media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_subdb/gsref.subdb.aspergillus_niger.minimap2.paf',
 'clavispora_lusitaniae

In [28]:
db_fn = new_db_fn
new_db_mapping_fn = {}
for species, fasta_fn in sub_reads_fn.items():
    tmp_out = dbases_fn[db_fn]
    db_name = os.path.basename(db_fn).replace('.fasta', '')
    out_fn = os.path.join(tmp_out, F"{db_name}.{species}.minimap2.paf")
    new_db_mapping_fn[species] = out_fn
    minimapmapping(fasta_fn, db_fn, out_fn)

(0, "[M::mm_idx_gen::0.007*1.28] collected minimizers\n[M::mm_idx_gen::0.009*2.48] sorted minimizers\n[M::main::0.009*2.47] loaded/built the index for 44 target sequence(s)\n[M::mm_mapopt_update::0.011*2.31] mid_occ = 45\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 44\n[M::mm_idx_stat::0.012*2.20] distinct minimizers: 9696 (71.01% are singletons); average occurrences: 2.462; average spacing: 5.428\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/puccinia_striiformis-tritici.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.puccinia_striiformis-tritici.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/puccinia_striiformis-tritici.15000.fasta\n[M::main] Real time: 0.014 sec; CPU: 0.028 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.009*1.20

(0, "[M::mm_idx_gen::0.009*1.19] collected minimizers\n[M::mm_idx_gen::0.012*2.32] sorted minimizers\n[M::main::0.012*2.31] loaded/built the index for 44 target sequence(s)\n[M::mm_mapopt_update::0.014*2.20] mid_occ = 45\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 44\n[M::mm_idx_stat::0.015*2.12] distinct minimizers: 9696 (71.01% are singletons); average occurrences: 2.462; average spacing: 5.428\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.penicillium_chrysogenum.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/penicillium_chrysogenum.15000.fasta\n[M::main] Real time: 0.017 sec; CPU: 0.033 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.008*1.25] collected min

(0, "[M::mm_idx_gen::0.008*1.20] collected minimizers\n[M::mm_idx_gen::0.011*2.26] sorted minimizers\n[M::main::0.011*2.25] loaded/built the index for 44 target sequence(s)\n[M::mm_mapopt_update::0.012*2.15] mid_occ = 45\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 44\n[M::mm_idx_stat::0.013*2.08] distinct minimizers: 9696 (71.01% are singletons); average occurrences: 2.462; average spacing: 5.428\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/yamadazyma_mexicana.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.yamadazyma_mexicana.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/yamadazyma_mexicana.15000.fasta\n[M::main] Real time: 0.015 sec; CPU: 0.028 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.010*1.16] collected minimizers\n[M:

(0, "[M::mm_idx_gen::0.006*1.26] collected minimizers\n[M::mm_idx_gen::0.009*2.60] sorted minimizers\n[M::main::0.009*2.59] loaded/built the index for 44 target sequence(s)\n[M::mm_mapopt_update::0.010*2.42] mid_occ = 45\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 44\n[M::mm_idx_stat::0.011*2.29] distinct minimizers: 9696 (71.01% are singletons); average occurrences: 2.462; average spacing: 5.428\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/oculimacula_yallundae-ccl031.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.oculimacula_yallundae-ccl031.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/oculimacula_yallundae-ccl031.15000.fasta\n[M::main] Real time: 0.013 sec; CPU: 0.028 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.006*1.28

(0, "[M::mm_idx_gen::0.011*1.13] collected minimizers\n[M::mm_idx_gen::0.014*2.05] sorted minimizers\n[M::main::0.014*2.05] loaded/built the index for 44 target sequence(s)\n[M::mm_mapopt_update::0.015*1.98] mid_occ = 45\n[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 44\n[M::mm_idx_stat::0.016*1.94] distinct minimizers: 9696 (71.01% are singletons); average occurrences: 2.462; average spacing: 5.428\nERROR: failed to open file '/media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/candida_albicans.15000.fasta'\n[M::main] Version: 2.17-r941\n[M::main] CMD: minimap2 -x map-ont -t 6 -o /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref_db/gsref.db.candida_albicans.minimap2.paf /media/MassStorage/tmp/TE/honours/analysis/Mapping/gsref.db.fasta /media/MassStorage/tmp/TE/honours/analysis/Mapping/MC_READS/candida_albicans.15000.fasta\n[M::main] Real time: 0.018 sec; CPU: 0.032 sec; Peak RSS: 0.005 GB")
(0, "[M::mm_idx_gen::0.011*1.11] collected minimizers\n[M::mm_idx_g

### Look at mapping results

In [29]:
def mapping_results(fn, species):
    min_header = ['qseqid', 'qlen', 'qstart', 'qstop', 'strand', 'tname', 'tlen', 'tstart', 'tend', 'nmatch', 'alen', 'mquality']
    tmp_df = pd.read_csv(fn, sep='\t', header = None, usecols=[x for x in range(0,12)], names=min_header)
    sub_df = tmp_df[tmp_df['mquality'] == tmp_df.groupby('qseqid')['mquality'].transform(max)].reset_index(drop=True)
    sub_df = sub_df[tmp_df['nmatch'] == tmp_df.groupby('qseqid')['nmatch'].transform(max)].reset_index(drop=True)
    hit_series = pd.Series(sub_df.groupby('tname')['mquality'].count().tolist()/sub_df.groupby('tname')['mquality'].count().sum(),
                      sub_df.groupby('tname')['mquality'].count().index)
    hit_series.sort_values(ascending=False, inplace=True)
    print('##########\n')
    print(F"This was the query species: {species}\n")
    print(F"These are the results:")
    print(hit_series,'\n')

In [30]:
###this is running the reads against the full database
for species, hit_fn in new_db_mapping_fn.items():
    mapping_results(hit_fn, species)

##########

This was the query species: puccinia_striiformis-tritici

These are the results:
Series([], dtype: float64) 

##########

This was the query species: zymoseptoria_tritici

These are the results:
Series([], dtype: float64) 

##########

This was the query species: pyrenophora_tritici-repentis

These are the results:
Series([], dtype: float64) 

##########

This was the query species: fusarium_oxysporum

These are the results:
Series([], dtype: float64) 

##########

This was the query species: tuber_brumale

These are the results:
Series([], dtype: float64) 

##########

This was the query species: cortinarius_globuliformis

These are the results:
Series([], dtype: float64) 

##########

This was the query species: aspergillus_niger

These are the results:
Series([], dtype: float64) 

##########

This was the query species: clavispora_lusitaniae

These are the results:
Series([], dtype: float64) 

##########

This was the query species: kluyveromyces_unidentified

These are 

In [ ]:
###this is running against a database that have ['Candida_orthopsilosis', 'Candida_metapsilosis', 'Aspergillus_niger'] deleted
for species, hit_fn in sub_db_mapping_fn.items():
    mapping_results(hit_fn, species)